In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
import pickle
import logging
import sys
from sklearn import preprocessing
from datetime import datetime as dt
from keras.models import load_model
sys.path.append('./src/assets/')
sys.path.append('./src')
from getData.extract import get_forex
from processData.processing import setup_data, get_indicators


pd.options.display.max_columns = None

# Setup logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO,
    #filename='log.txt'
)

# constantes para extraer datos
logging.info('************* Inicio **************')
candleformat = 'bidask'
instrument = 'USD_JPY'
instruments = ['USD_JPY', 
               'USB02Y_USD',
               'USB05Y_USD',
               'USB10Y_USD', 
               'USB30Y_USD',
               'UK100_GBP',
               'UK10YB_GBP',
               'JP225_USD',
               'HK33_HKD',
               'EU50_EUR',
               'DE30_EUR',
               'DE10YB_EUR',
               'WTICO_USD',
               'US30_USD',
               'SPX500_USD',
               'NL25_EUR',
               'FR40_EUR',
               'AU200_AUD',
               'US2000_USD',
               'XAU_USD',
               'AUD_USD',
               'GBP_USD',
               'USD_CAD',
               'USD_CHF',
               'EUR_USD']
               
granularity = 'H1'
start = str(datetime.datetime.now() + datetime.timedelta(days=-2))[:10]
end = str(dt.now())[:10]
freq = 'D'
trading = True

# variables para High
logging.info('************* Cargando Variables **************')
variablesh = pd.read_csv('./src/assets/variables/variablesHigh.csv')
variablesh = list(variablesh['0'].values)

# variables para Low
variablesl = pd.read_csv('./src/assets/variables/variablesLow.csv')
variablesl = list(variablesl['0'].values)


# carga de modelos
logging.info('************* Cargando Modelos  Gradient Boosting (High)**************')
gbH_6 = pickle.load(open('./src/assets/models/gbHigh-6.h5', 'rb'))
gbH_5 = pickle.load(open('./src/assets/models/gbHigh-5.h5', 'rb'))
gbH_4 = pickle.load(open('./src/assets/models/gbHigh-4.h5', 'rb'))
gbH_3 = pickle.load(open('./src/assets/models/gbHigh-3.h5', 'rb'))
gbH_2 = pickle.load(open('./src/assets/models/gbHigh-2.h5', 'rb'))
gbH_1 = pickle.load(open('./src/assets/models/gbHigh-1.h5', 'rb'))
gbH0 = pickle.load(open('./src/assets/models/gbHigh0.h5', 'rb'))
gbH1 = pickle.load(open('./src/assets/models/gbHigh1.h5', 'rb'))
gbH2 = pickle.load(open('./src/assets/models/gbHigh2.h5', 'rb'))
gbH3 = pickle.load(open('./src/assets/models/gbHigh3.h5', 'rb'))
gbH4 = pickle.load(open('./src/assets/models/gbHigh4.h5', 'rb'))
gbH5 = pickle.load(open('./src/assets/models/gbHigh5.h5', 'rb'))
gbH6 = pickle.load(open('./src/assets/models/gbHigh6.h5', 'rb'))



logging.info('************* Cargando Modelos  Gradient Boosting (Low)**************')
gbL_6 = pickle.load(open('./src/assets/models/gbLow-6.h5', 'rb'))
gbL_5 = pickle.load(open('./src/assets/models/gbLow-5.h5', 'rb'))
gbL_4 = pickle.load(open('./src/assets/models/gbLow-4.h5', 'rb'))
gbL_3 = pickle.load(open('./src/assets/models/gbLow-3.h5', 'rb'))
gbL_2 = pickle.load(open('./src/assets/models/gbLow-2.h5', 'rb'))
gbL_1 = pickle.load(open('./src/assets/models/gbLow-1.h5', 'rb'))
gbL0 = pickle.load(open('./src/assets/models/gbLow0.h5', 'rb'))
gbL1 = pickle.load(open('./src/assets/models/gbLow1.h5', 'rb'))
gbL2 = pickle.load(open('./src/assets/models/gbLow2.h5', 'rb'))
gbL3 = pickle.load(open('./src/assets/models/gbLow3.h5', 'rb'))
gbL4 = pickle.load(open('./src/assets/models/gbLow4.h5', 'rb'))
gbL5 = pickle.load(open('./src/assets/models/gbLow5.h5', 'rb'))
gbL6 = pickle.load(open('./src/assets/models/gbLow6.h5', 'rb'))

# descarga de datos
logging.info('************* Descargando datos **************')
gf = get_forex(instrument, instruments, granularity, start, end, candleformat, freq, trading)

sd = setup_data(gf,
                instrument=instrument,
                pricediff=True,
                log=True,
                trading=True)

# indicadores que se añaden al modelo
logging.info('************* Creando Indicadores **************')
processeddf = get_indicators(sd, 
                             instrument, 
                             column='{}_closeBid'.format(instrument), 
                             wind=10, 
                             bidask='Bid') 
processeddf = get_indicators(processeddf, 
                             instrument, 
                             column='{}_closeAsk'.format(instrument), 
                             wind=10, 
                             bidask='Ask')

processeddf = processeddf.fillna(method='ffill')
processeddf = processeddf.fillna(method='bfill')

# carga de escaladores
logging.info('************* Escalando Datos **************')
Xh = processeddf[variablesh]
scaler = pickle.load(open('./src/assets/models/scalerH', 'rb'))
Xh = scaler.transform(Xh)


Xl = processeddf[variablesl]
scaler = pickle.load(open('./src/assets/models/scalerL', 'rb'))
Xl = scaler.transform(Xl)


Using TensorFlow backend.
10/05/2019 12:11:18 AM - INFO - ************* Inicio **************
10/05/2019 12:11:18 AM - INFO - ************* Cargando Variables **************
10/05/2019 12:11:18 AM - INFO - ************* Cargando Modelos  Gradient Boosting (High)**************
10/05/2019 12:11:18 AM - INFO - ************* Cargando Modelos  Gradient Boosting (Low)**************
10/05/2019 12:11:18 AM - INFO - ************* Descargando datos **************
10/05/2019 12:11:18 AM - INFO - USD_JPY
10/05/2019 12:11:18 AM - INFO - USB02Y_USD
10/05/2019 12:11:19 AM - INFO - USB05Y_USD
10/05/2019 12:11:19 AM - INFO - USB10Y_USD
10/05/2019 12:11:19 AM - INFO - USB30Y_USD
10/05/2019 12:11:19 AM - INFO - UK100_GBP
10/05/2019 12:11:19 AM - INFO - UK10YB_GBP
10/05/2019 12:11:19 AM - INFO - JP225_USD
10/05/2019 12:11:19 AM - INFO - HK33_HKD
10/05/2019 12:11:19 AM - INFO - EU50_EUR
10/05/2019 12:11:20 AM - INFO - DE30_EUR
10/05/2019 12:11:20 AM - INFO - DE10YB_EUR
10/05/2019 12:11:20 AM - INFO - WTICO

In [2]:
# predicciones
logging.info('************* Haciendo Predicciones **************')

models = {}
models['Xh_gbH-6'] = gbH_6.predict_proba(Xh)[:,1]
models['Xh_gbH-5'] = gbH_5.predict_proba(Xh)[:,1]
models['Xh_gbH-4'] = gbH_4.predict_proba(Xh)[:,1]
models['Xh_gbH-3'] = gbH_3.predict_proba(Xh)[:,1]
models['Xh_gbH-2'] = gbH_2.predict_proba(Xh)[:,1]
models['Xh_gbH-1'] = gbH_1.predict_proba(Xh)[:,1]
models['Xh_gbH0']= gbH0.predict_proba(Xh)[:,1]
models['Xh_gbH1']= gbH1.predict_proba(Xh)[:,1]
models['Xh_gbH2']= gbH2.predict_proba(Xh)[:,1]
models['Xh_gbH3']= gbH3.predict_proba(Xh)[:,1]
models['Xh_gbH4']= gbH4.predict_proba(Xh)[:,1]
models['Xh_gbH5']= gbH5.predict_proba(Xh)[:,1]
models['Xh_gbH6']= gbH6.predict_proba(Xh)[:,1]

models['Xl_gbl-6'] = gbL_6.predict_proba(Xl)[:,1]
models['Xl_gbl-5'] = gbL_5.predict_proba(Xl)[:,1]
models['Xl_gbl-4'] = gbL_4.predict_proba(Xl)[:,1]
models['Xl_gbl-3'] = gbL_3.predict_proba(Xl)[:,1]
models['Xl_gbl-2'] = gbL_2.predict_proba(Xl)[:,1]
models['Xl_gbl-1'] = gbL_1.predict_proba(Xl)[:,1]
models['Xl_gbl0']= gbL0.predict_proba(Xl)[:,1]
models['Xl_gbl1']= gbL1.predict_proba(Xl)[:,1]
models['Xl_gbl2']= gbL2.predict_proba(Xl)[:,1]
models['Xl_gbl3']= gbL3.predict_proba(Xl)[:,1]
models['Xl_gbl4']= gbL4.predict_proba(Xl)[:,1]
models['Xl_gbl5']= gbL5.predict_proba(Xl)[:,1]
models['Xl_gbl6']= gbL6.predict_proba(Xl)[:,1]

Xh_gbH_6 = gbH_6.predict_proba(Xh)[:,1]
Xh_gbH_5 = gbH_5.predict_proba(Xh)[:,1]
Xh_gbH_4 = gbH_4.predict_proba(Xh)[:,1]
Xh_gbH_3 = gbH_3.predict_proba(Xh)[:,1]
Xh_gbH_2 = gbH_2.predict_proba(Xh)[:,1]
Xh_gbH_1 = gbH_1.predict_proba(Xh)[:,1]
Xh_gbH0 = gbH0.predict_proba(Xh)[:,1]
Xh_gbH1 = gbH1.predict_proba(Xh)[:,1]
Xh_gbH2 = gbH2.predict_proba(Xh)[:,1]
Xh_gbH3 = gbH3.predict_proba(Xh)[:,1]
Xh_gbH4 = gbH4.predict_proba(Xh)[:,1]
Xh_gbH5 = gbH5.predict_proba(Xh)[:,1]
Xh_gbH6 = gbH6.predict_proba(Xh)[:,1]

Xl_gbl_6 = gbL_6.predict_proba(Xl)[:,1]
Xl_gbl_5 = gbL_5.predict_proba(Xl)[:,1]
Xl_gbl_4 = gbL_4.predict_proba(Xl)[:,1]
Xl_gbl_3 = gbL_3.predict_proba(Xl)[:,1]
Xl_gbl_2 = gbL_2.predict_proba(Xl)[:,1]
Xl_gbl_1 = gbL_1.predict_proba(Xl)[:,1]
Xl_gbl0 = gbL0.predict_proba(Xl)[:,1]
Xl_gbl1 = gbL1.predict_proba(Xl)[:,1]
Xl_gbl2 = gbL2.predict_proba(Xl)[:,1]
Xl_gbl3 = gbL3.predict_proba(Xl)[:,1]
Xl_gbl4 = gbL4.predict_proba(Xl)[:,1]
Xl_gbl5 = gbL5.predict_proba(Xl)[:,1]
Xl_gbl6 = gbL6.predict_proba(Xl)[:,1]

preds_buy = {
'Xh_gbH_6':Xh_gbH_6[-2],
'Xh_gbH_5':Xh_gbH_5[-2],
'Xh_gbH_4':Xh_gbH_4[-2],
'Xh_gbH_3':Xh_gbH_3[-2],
'Xh_gbH_2':Xh_gbH_2[-2],
'Xh_gbH_1':Xh_gbH_1[-2],
'Xh_gbH0':Xh_gbH0[-2],
'Xh_gbH1':Xh_gbH1[-2],
'Xh_gbH2':Xh_gbH2[-2],
'Xh_gbH3':Xh_gbH3[-2],
'Xh_gbH4':Xh_gbH4[-2],
'Xh_gbH5':Xh_gbH5[-2],
'Xh_gbH6':Xh_gbH6[-2]
}

preds_sell= {
'Xl_gbl_6': Xl_gbl_6[-2],
'Xl_gbl_5': Xl_gbl_5[-2],
'Xl_gbl_4': Xl_gbl_4[-2],
'Xl_gbl_3': Xl_gbl_3[-2],
'Xl_gbl_2': Xl_gbl_2[-2],
'Xl_gbl_1': Xl_gbl_1[-2],
'Xl_gbl0': Xl_gbl0[-2],
'Xl_gbl1': Xl_gbl1[-2],
'Xl_gbl2': Xl_gbl2[-2],
'Xl_gbl3': Xl_gbl3[-2],
'Xl_gbl4': Xl_gbl4[-2],
'Xl_gbl5': Xl_gbl5[-2],
'Xl_gbl6': Xl_gbl6[-2]
}

pricediff = True
instrument = 'USD_JPY'

Actuals = ['HHLL_LogDiff {}_highAsk'.format(instrument),
           'HHLL_LogDiff {}_highBid'.format(instrument),
           'HHLL_LogDiff {}_lowAsk'.format(instrument),
           'HHLL_LogDiff {}_lowBid'.format(instrument)]

Responses = ['future diff high',
             'future diff low']

prices = [i.replace('HHLL_LogDiff ', '') for i in Actuals]
prices.append('{}_date'.format(instrument))

fxcm = gf[prices]
fxcm['{}_date'.format(instrument)] = fxcm['{}_date'.format(instrument)].astype(str)
fxcm['{}_date'.format(instrument)] = fxcm['{}_date'.format(instrument)].str[:13]

fxcm = fxcm.drop(0)

variableshhll = pd.read_csv('./src/assets/variables/variablesHigh.csv', index_col=0)

10/05/2019 12:11:24 AM - INFO - ************* Haciendo Predicciones **************


In [3]:
current_open_ask = gf['USD_JPY_openAsk'].iloc[-1].round(3)
current_open_bid = gf['USD_JPY_openBid'].iloc[-1].round(3)
print(f'current_open_ask: {current_open_ask} current_open_bid:{current_open_bid}')

current_open_ask: 106.976 current_open_bid:106.876


In [4]:
previous_high_ask = gf['USD_JPY_highAsk'].iloc[-2].round(3)

In [5]:
take_profit_buy = [((1+i/(10000.00))*previous_high_ask).round(3) for i in range(-6, 7)]

In [6]:
previous_low_bid = gf['USD_JPY_lowBid'].iloc[-2].round(3)

In [7]:
take_profit_sell = [((1-i/(10000.00))*previous_low_bid).round(3) for i in range(-6, 7)]

In [8]:
indx = 'Buy'
df_buy = pd.DataFrame({'Open': current_open_ask,
                       'Probability':list(preds_buy.values()),
                       'Take Profit': take_profit_buy,
                       'Portfolio Gain': np.zeros(13),
                       'Utility Gain': np.zeros(13)},
                  index=[indx + '-6',
                         indx + '-5',
                         indx + '-4',
                         indx + '-3',
                         indx + '-2',
                         indx + '-1',
                         indx + '0',
                         indx + '1',
                         indx + '2',
                         indx + '3',
                         indx + '4',
                         indx + '5',
                         indx + '6'])
indx = 'Sell'
df_sell = pd.DataFrame({'Open': current_open_bid,
                        'Probability':list(preds_sell.values()),
                        'Take Profit': take_profit_sell,
                       'Portfolio Gain': np.zeros(13),
                       'Utility Gain': np.zeros(13)},
                  index=[indx + '-6',
                         indx + '-5',
                         indx + '-4',
                         indx + '-3',
                         indx + '-2',
                         indx + '-1',
                         indx + '0',
                         indx + '1',
                         indx + '2',
                         indx + '3',
                         indx + '4',
                         indx + '5',
                         indx + '6'])


In [9]:
df_buy

,Open,Probability,Take Profit,Portfolio Gain,Utility Gain
Buy-6,106.976,0.950069,106.908,0.0,0.0
Buy-5,106.976,0.916141,106.919,0.0,0.0
Buy-4,106.976,0.862574,106.929,0.0,0.0
Buy-3,106.976,0.723561,106.940,0.0,0.0
Buy-2,106.976,0.609604,106.951,0.0,0.0
Buy-1,106.976,0.524133,106.961,0.0,0.0
Buy0,106.976,0.472918,106.972,0.0,0.0
Buy1,106.976,0.401264,106.983,0.0,0.0
Buy2,106.976,0.355961,106.993,0.0,0.0
Buy3,106.976,0.316902,107.004,0.0,0.0


In [10]:
df_sell

,Open,Probability,Take Profit,Portfolio Gain,Utility Gain
Sell-6,106.876,0.604807,106.889,0.0,0.0
Sell-5,106.876,0.478198,106.878,0.0,0.0
Sell-4,106.876,0.409663,106.868,0.0,0.0
Sell-3,106.876,0.298924,106.857,0.0,0.0
Sell-2,106.876,0.238718,106.846,0.0,0.0
Sell-1,106.876,0.187630,106.836,0.0,0.0
Sell0,106.876,0.155581,106.825,0.0,0.0
Sell1,106.876,0.122788,106.814,0.0,0.0
Sell2,106.876,0.098297,106.804,0.0,0.0
Sell3,106.876,0.090835,106.793,0.0,0.0
